In [ ]:
from kafka import KafkaConsumer
import pandas as pd

consumer = KafkaConsumer('cyan3', bootstrap_servers=['localhost:9092'])

ticker_symbol = 'AAPL'
window = 20
threshold = 2.0

prices = pd.DataFrame(columns=['Close'])
retested_upper_band = False

for message in consumer:
    data = message.value.decode('utf-8').split(',')
    close_price = float(data[3])
    prices = prices.append({'Close': close_price}, ignore_index=True)

    if len(prices) < window:
        continue

    # Calculate the rolling mean and standard deviation
    rolling_mean = prices['Close'].rolling(window).mean()
    rolling_std = prices['Close'].rolling(window).std()

    # Calculate the upper and lower Bollinger Bands
    upper_band = rolling_mean + threshold * rolling_std
    lower_band = rolling_mean - threshold * rolling_std

    # Generate insights based on the current price relative to the Bollinger Bands
    if close_price > upper_band.iloc[-1]:
        if not retested_upper_band:
            retested_upper_band = True
        else:
            print(f"{ticker_symbol} has retested the upper Bollinger Band and is expected to go down from {close_price}")
    elif close_price < lower_band.iloc[-1]:
        retested_upper_band = False
        print(f"{ticker_symbol} is oversold with a price of {close_price}")